# Predicting lung cancer survival time by OWKIN

### Problem

- supervised survival prediction problem
- predict the survival time of a patient (remaining days to live) from one three-dimensional CT scan (grayscale image) and a set of pre-extracted quantitative imaging features, as well as clinical data

### Import

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

### Data

- x_train : data_Q0G7b5t
- y_train : output_VSVxRFU.csv
- x_test : data_9Cbe5hx

In [4]:
data_folder_path = "../data"
training_folder_path = os.path.join(data_folder_path, "data_Q0G7b5t")
test_folder_path = os.path.join(data_folder_path, "data_9Cbe5hx")

training_ct_scan_names = [os.path.join(root,file_name) for root,_,file_names in os.walk(training_folder_path) for file_name in file_names if file_name.endswith('.npz')]
test_ct_scan_names = [os.path.join(root,file_name) for root,_,file_names in os.walk(test_folder_path) for file_name in file_names if file_name.endswith('.npz')]

print("Number of training ct scans : {}".format(len(training_ct_scan_names)))
print("Number of test ct scans : {}".format(len(test_ct_scan_names)))

training_features_path = os.path.join(training_folder_path, "features")
test_features_path = os.path.join(test_folder_path, "features")

Number of training ct scans : 300
Number of test ct scans : 125


In [16]:
archive = np.load(training_ct_scan_names[0])
scan = archive['scan']
mask = archive['mask']
# scan.shape equals mask.shape

In [18]:
train_output = pd.read_csv(os.path.join(data_folder_path, "output_VSVxRFU.csv"), index_col=0)
p0 = train_output.loc[202]
print("p0.Event", p0.Event) # prints 1 or 0
print("p0.SurvivalTime", p0.SurvivalTime)
# prints time to event (time to death or time to last known alive) in days

p0.Event 0
p0.SurvivalTime 1378


### Interpretation

(`1=death observed`, `0=escaped from study`)

### Load training data

In [36]:
file_name = os.path.join(training_features_path, "clinical_data.csv")
df_training_clinical_data = pd.read_csv(file_name, delimiter=',')
print("Nb rows in df_training_clinical_data : {}".format(len(df_training_clinical_data)))

file_name = os.path.join(training_features_path, "radiomics.csv")
df_training_radiomics = pd.read_csv(file_name, delimiter=',', header=[0,1,2])
print("Nb rows in df_training_radiomics : {}".format(len(df_training_radiomics)))

Nb rows in df_training_clinical_data : 300
Nb rows in df_training_radiomics : 300


### clinical_data.csv

In [38]:
df_training_clinical_data.sample(5)

,PatientID,Histology,Mstage,Nstage,SourceDataset,Tstage,age
145,406,Adenocarcinoma,0,0,l2,1,61.0000
248,32,Adenocarcinoma,0,0,l2,1,74.0000
80,30,large cell,0,2,l1,2,85.1116
126,23,Adenocarcinoma,0,2,l2,3,43.0000
153,18,NaN,0,0,l1,1,NaN


#### Are there NaN values in df_training_radiomics ?

In [46]:
df_training_clinical_data.isnull().sum()

PatientID         0
Histology        20
Mstage            0
Nstage            0
SourceDataset     0
Tstage            0
age              16
dtype: int64

### Remark

There are NaN values Histology and age columns. We will not use these in our study so no problem.

### radiomics.csv

In [39]:
df_training_radiomics.sample(5)

Unnamed: 0_level_0                       shape  \
    Unnamed: 0_level_1 original_shape_Compactness1   
             PatientID          Unnamed: 1_level_2   
145                406                    0.026370   
16                 185                    0.018633   
125                152                    0.033609   
288                375                    0.027013   
164                 25                    0.029582   

                                                                  \
    original_shape_Compactness2 original_shape_Maximum3DDiameter   
             Unnamed: 2_level_2               Unnamed: 3_level_2   
145                    0.247075                        28.513155   
16                     0.123362                        58.898217   
125                    0.401332                        26.570661   
288                    0.259260                        31.638584   
164                    0.310925                        25.884358   

                                                                     \
    original_shape_SphericalDisproportion original_shape_Sphericity   
                       Unnamed: 4_level_2        Unnamed: 5_level_2   
145                              1.593642                  0.627494   
16                               2.008814                  0.497806   
125                              1.355706                  0.737623   
288                              1.568271                  0.637645   
164                              1.476096                  0.677463   

                                                                  \
    original_shape_SurfaceArea original_shape_SurfaceVolumeRatio   
            Unnamed: 6_level_2                Unnamed: 7_level_2   
145                1523.109185                          0.548210   
16                 6376.235249                          0.379187   
125                1294.810031                          0.466521   
288                2109.807794                          0.454712   
164                1613.790265                          0.474761   

                                               firstorder  ...  \
    original_shape_VoxelVolume original_firstorder_Energy  ...   
            Unnamed: 8_level_2         Unnamed: 9_level_2  ...   
145                     2809.0               8.577067e+08  ...   
16                     16937.0               1.608007e+09  ...   
125                     2799.0               1.492792e+08  ...   
288                     4671.0               9.850466e+08  ...   
164                     3431.0               1.535280e+09  ...   

                          textural                                        \
    original_glrlm_LongRunEmphasis original_glrlm_GrayLevelNonUniformity   
               Unnamed: 44_level_2                   Unnamed: 45_level_2   
145                       1.350421                             77.557194   
16                        1.257524                            658.032193   
125                       1.778461                            117.206529   
288                       1.278623                            144.651090   
164                       1.168042                            101.532610   

                                                                        \
    original_glrlm_RunLengthNonUniformity original_glrlm_RunPercentage   
                      Unnamed: 46_level_2          Unnamed: 47_level_2   
145                           2183.839359                     0.910754   
16                           13707.746024                     0.927864   
125                           1805.694745                     0.840025   
288                           3750.591437                     0.924098   
164                           2954.287460                     0.949712   

                                            \
    original_glrlm_LowGrayLevelRunEmphasis   
                       Unnamed: 48_level_2   
145                               0.012103 

#### Are there NaN values in df_training_radiomics ?

In [44]:
df_training_radiomics.isnull().sum().sum()

0

### Remark

There are no NaN values in df_training_radiomics.

### Baseline model for survival regression on NSCLC clinical data : Cox proportional hazard (Cox-PH) model

This baseline is trained on a selection of features from both clinical data file and radiomics file. A Cox-PH model was fitted on

- 1 - Tumor sphericity, a measure of the roundness of the shape of the tumor region relative to a sphere, regardless its dimensions (size).
- 2 - The tumor's surface to volume ratio is a measure of the compactness of the tumor, related to its size.
- 3 - The tumor's maximum 3d diameter The biggest diameter measurable from the tumor volume
- 4 - The dataset of origin
- 5 - The N-tumoral stage grading of the tumor describing nearby (regional) lymph nodes involved
- 6 - The tumor's joint entropy, specifying the randomness in the image pixel values
- 7 - The tumor's inverse different, a measure of the local homogeneity of the tumor
- 8 - The tumor's inverse difference moment is another measurement of the local homogeneity of the tumor

### Name of variables

- 1 - original_shape_Sphericity
- 2 - original_shape_SurfaceVolumeRatio
- 3 - original_shape_Maximum3DDiameter
- 4 - l1 (0) or l2 (1)
- 5 - Nstage
- 6 - original_firstorder_Entropy
- 7 - inverse difference (original_glcm_Id)
- 8 - inverse difference moment (original_glcm_Idm) (according to [here](https://static-content.springer.com/esm/art%3A10.1038%2Fncomms5006/MediaObjects/41467_2014_BFncomms5006_MOESM716_ESM.pdf), ctr+F IDMN and [here](https://github.com/cerr/CERR/wiki/GLCM_global_features))

### Remark

Variables used in the baseline use quantitve and qualitive variables. (dataset of origin (l1 or l2)) makes no sens 

## $\color{red}{\text{To be continued}}$